In [ ]:
import laspy
import fiona
import numpy as np
import itertools
import pandas
import laspy
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import open3d as o3d
import os
from os import listdir,mkdir




In [ ]:
!find . -name ".DS_Store" -delete
shape = fiona.open("/Users/parrot/Envs/Lidar_Repo/resources/2019_angkormap/Master/AngkorMaster.shp/archaeology.shp")
#voxel size
voxel_size=1
#number of sectors/chunks in one direction 
cropnum=10
directory="testdata"
child_directory="las"
parent_dir="/Users/parrot/Envs/Lidar_Repo/data/"
path = os.path.join(parent_dir, child_directory, directory) 
for files in listdir(path):
    print(files)
    fullpath=os.path.join(path,files)
    print(fullpath)
    inFile = laspy.file.File(fullpath, mode = "r")
    splitat = 10
    s =files[0:splitat]
    suffix = '.xyz'
    txtfullpath=os.path.join(parent_dir, "txt/testdata",s + suffix)
    datasetfull = np.vstack([inFile.X/100, inFile.Y/100,inFile.Z/100]).transpose()
    np.savetxt(txtfullpath, datasetfull)
    pcd = o3d.io.read_point_cloud(txtfullpath)
    #downsampling pc
    downpcd = pcd.voxel_down_sample(voxel_size)
    dataset = o3d.np.asarray(downpcd.points)[:,0:2]
    zval=o3d.np.asarray(downpcd.points)[:,2]
    splitat = 5
    l, r =files[1:splitat], files[splitat:10]
    xmin=100*int(l)
    ymin=100*int(r)
    xmax=100*(int(l)+5)
    ymax=100*(int(r)+5)
    #find which polygons are inside boundary box of the pc 
    hits=list(shape.items(bbox=(xmin, ymin, xmax, ymax)))
    
    zval=zval.tolist()
    fullarray=[]
    fullzeroarray=[]
    #iterate over the polygons in the pointcloud
    for m in range(len(hits)-1):
        
        polynum=m
        
        #iterate over the coordinates of the polygon
        for j in range(len(hits[polynum][1]['geometry']['coordinates'])):
        
            print('polygon with this is id is th',j)
            
            unpacklist=hits[polynum][1]['geometry']['coordinates'][j]
               
            if len(unpacklist)==1:
                
                unpacklist=unpacklist[0]
                
            
            polygon=Polygon(unpacklist)       
            array = []
            zeroarray=[]
            #find which pc points lie inside these polygons
            for i in range(np.shape(dataset)[0]):
                point=Point(dataset[i,:])
                if point.within(polygon)==True:
                    array.append(i)
               
        fullarray.append(array)
    ################    
    ################
    
    dp=dataset.tolist()
    getgen=[]
    for ida in range(len(fullarray)):
        
        for idx,l in enumerate(fullarray[ida]):
            
    #append the feature type of the polygon to the 3d points 
            dp[l].extend((zval[l],hits[ida][1]['properties']['type']))

    for i in range(len(dp)):
        if len(dp[i])==2:
            dp[i].extend((zval[i],0))


    splitat = 5
    l, r =files[1:splitat], files[splitat:10]

    #define the extent of the sectors/chunks 
    xspace=(xmax-xmin)/cropnum
    yspace=(ymax-ymin)/cropnum
    ls=[]

    #find which sectors actually contain points with labels
    for k in range(len(dp)): 
        
        for i, j in itertools.product(range(cropnum-1), range(cropnum-1)):
            if (xmin+xspace*(i)<dp[k][0]<xmin+xspace*(i+1)) & (ymin+yspace*(j)<dp[k][1]<ymin+yspace*(j+1)) & (dp[k][3]!=0):
                ls.append((i,j)) if (i,j) not in ls else ls
    #print(ls)
    getem=[]      
#extract only the points that lie inside sectors with labels

    xspace=(xmax-xmin)/cropnum
    yspace=(ymax-ymin)/cropnum
    for s, t in ls:
        get=[]
        for k in range(len(dp)):
            
            if (xmin+xspace*(s)<dp[k][0]<xmin+xspace*(s+1)) & (ymin+yspace*(t)<dp[k][1]<ymin+yspace*(t+1)):
                get.append(dp[k])
                
        getem.append(get)
    

    getem_dpl = [[] for _ in range(lengetem)]

    #this section deals with points that have more than one label. 
    for ia in range(len(getem)):
        for ib in range(len(getem[ia])):
            if len(getem[ia][ib])<5:
            
                getem_dpl[ia].insert(ib,getem[ia][ib][0:4])

            else:
                ls=list(range(2,len(getem[ia][ib]),2))
             
                for s,k in enumerate(ls):
                    
                    lr=[0,1,k,k+1]
                    a=np.array(getem[ia][ib])[lr]
                    getem_dpl[ia].append(list(a))

    directory2="testdataout"
    child_directory2="txt"
    parent_dir2="/Users/parrot/Envs/Lidar_Repo/data/"
    path2 = os.path.join(parent_dir2, child_directory2, directory2) 

    for i in range(len(getem_dpl)):
        
        splitat = 10
        s =files[0:splitat]
        print(s)
        suffix = '.xyz'
        txtfullpath2=os.path.join(parent_dir2, "txt/testdataout",s +'_'+str(i) + suffix)
        np.savetxt(txtfullpath2, getem_dpl[i])

 
################
################
